<a href="https://colab.research.google.com/github/mshahmeer1/Radiological-phenotypes-of-ARDS/blob/main/Data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import os

pd.set_option('display.max_colwidth',100000) #https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from google.colab import files

In [ ]:
#user authentication
auth.authenticate_user()

In [ ]:
# Set up the project ID
# Note that the physionet-data project is for data hosting only.
project_id = 'august-tangent-380700'
os.environ['august-tangent-380700'] = project_id

In [ ]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      configuration={'query': {
          'useLegacySql': False
      }})

In [ ]:
query = ''' with mvp AS
(
with ce_stg1 as
(
  SELECT
      ce.subject_id
    , ce.stay_id
    , ce.charttime
    , CASE
        -- merge o2 flows into a single row
        WHEN itemid IN (223834, 227582, 224691) THEN 223834
      ELSE itemid END AS itemid
    , value
    , valuenum
    , valueuom
    , storetime
  FROM physionet-data.mimiciv_icu.chartevents ce
  WHERE ce.value IS NOT NULL
  AND ce.itemid IN
  (
      223834 -- o2 flow
    , 227582 -- bipap o2 flow
    , 224691 -- Flow Rate (L)
    -- additional o2 flow is its own column
    , 227287 -- additional o2 flow
  )
)
, ce_stg2 AS
(
  select
    ce.subject_id
    , ce.stay_id
    , ce.charttime
    , itemid
    , value
    , valuenum
    , valueuom
    -- retain only 1 row per charttime
    -- prioritizing the last documented value
    -- primarily used to subselect o2 flows
    , ROW_NUMBER() OVER (PARTITION BY subject_id, charttime, itemid ORDER BY storetime DESC) as rn
  FROM ce_stg1 ce
)
, o2 AS
(
    -- The below ITEMID can have multiple entires for charttime/storetime
    -- These are totally valid entries, and should be retained in derived tables.
    --   224181 -- Small Volume Neb Drug #1              | Respiratory             | Text       | chartevents
    -- , 227570 -- Small Volume Neb Drug/Dose #1         | Respiratory             | Text       | chartevents
    -- , 224833 -- SBT Deferred                          | Respiratory             | Text       | chartevents
    -- , 224716 -- SBT Stopped                           | Respiratory             | Text       | chartevents
    -- , 224740 -- RSBI Deferred                         | Respiratory             | Text       | chartevents
    -- , 224829 -- Trach Tube Type                       | Respiratory             | Text       | chartevents
    -- , 226732 -- O2 Delivery Device(s)                 | Respiratory             | Text       | chartevents
    -- , 226873 -- Inspiratory Ratio                     | Respiratory             | Numeric    | chartevents
    -- , 226871 -- Expiratory Ratio                      | Respiratory             | Numeric    | chartevents
    -- maximum of 4 o2 devices on at once
    SELECT
        subject_id
        , stay_id
        , charttime
        , itemid
        , value AS o2_device
    , ROW_NUMBER() OVER (PARTITION BY subject_id, charttime, itemid ORDER BY value) as rn
    FROM physionet-data.mimiciv_icu.chartevents
    WHERE itemid = 226732 
    AND value in ('Trach mask', 'Tracheostomy tube', 'Bipap mask', 'CPAP mask', 'Endotracheal tube', 'T-piece')-- oxygen delivery device(s)
)
, stg AS
(
    select
      COALESCE(ce.subject_id, o2.subject_id) AS subject_id
    , COALESCE(ce.stay_id, o2.stay_id) AS stay_id
    , COALESCE(ce.charttime, o2.charttime) AS charttime
    , COALESCE(ce.itemid, o2.itemid) AS itemid
    , ce.value
    , ce.valuenum
    , o2.o2_device
    , o2.rn
    from ce_stg2 ce
    FULL OUTER JOIN o2
      ON ce.subject_id = o2.subject_id
      AND ce.charttime = o2.charttime
    -- limit to 1 row per subject_id/charttime/itemid from ce_stg2
    WHERE ce.rn = 1
)
SELECT
    subject_id
    , MAX(stay_id) AS stay_id
    , charttime
    , MAX(CASE WHEN itemid = 223834 THEN valuenum ELSE NULL END) AS o2_flow
    , MAX(CASE WHEN itemid = 227287 THEN valuenum ELSE NULL END) AS o2_flow_additional
    -- ensure we retain all o2 devices for the patient
    , MAX(CASE WHEN rn = 1 THEN o2_device ELSE NULL END) AS o2_delivery_device_1
    , MAX(CASE WHEN rn = 2 THEN o2_device ELSE NULL END) AS o2_delivery_device_2
    , MAX(CASE WHEN rn = 3 THEN o2_device ELSE NULL END) AS o2_delivery_device_3
    , MAX(CASE WHEN rn = 4 THEN o2_device ELSE NULL END) AS o2_delivery_device_4
FROM stg
GROUP BY subject_id, charttime

)
SELECT
 *
 FROM mvp
 WHERE o2_delivery_device_1 IS NOT NULL
 OR o2_delivery_device_2 IS NOT NULL
 OR o2_delivery_device_3 IS NOT NULL
 OR o2_delivery_device_4 IS NOT NULL


'''

d= run_query(query)
d.head(5)

In [ ]:
d.to_csv('MV.csv')
files.download('MV.csv')

In [ ]:
query = ''' 

 SELECT 
   subject_id
    , stay_id
    , charttime
    , value
    , valuenum
    , valueuom
  FROM physionet-data.mimiciv_icu.chartevents 
  WHERE itemid = 220224
ORDER BY subject_id, stay_id, charttime
'''
pO2 = run_query(query)

pO2.head(5)


In [ ]:
pO2.to_csv('pO2.csv')
files.download('pO2.csv')

In [ ]:
query= '''
SELECT 
      subject_id
    , stay_id
    , charttime
    , value
    , valuenum
    , valueuom,
    CASE
      WHEN valuenum >= 21.0 THEN valuenum * 0.01
      WHEN valuenum >= 1.0 and valuenum < 21.0 THEN (20 + (4 * valuenum)) * 0.01
      WHEN valuenum < 1.0 AND valuenum > 0 THEN valuenum
      WHEN valuenum = 0.0 THEN 0.21
      ELSE NULL
    END as converted_fiO2,


  FROM physionet-data.mimiciv_icu.chartevents
  WHERE itemid = 223835
ORDER BY subject_id, stay_id, charttime
'''
fiO2 = run_query(query)

fiO2.head(5)


In [ ]:
fiO2.to_csv('fiO2.csv')
files.download('fiO2.csv')

In [ ]:
query= '''
SELECT * 

FROM physionet-data.mimiciv_icu.procedureevents
  WHERE itemid = 221216

ORDER BY subject_id, stay_id
'''
Xray = run_query(query)

Xray.head(5)


In [ ]:
Xray.to_csv('Xray.csv')
files.download('Xray.csv')

In [ ]:
query= '''
SELECT *

FROM `physionet-data.mimiciv_derived.age`

WHERE age >= 18
ORDER BY subject_id, hadm_id
'''
age = run_query(query)

age.head(5)

In [ ]:
age.to_csv('age.csv')
files.download('age.csv')

In [ ]:
query= '''
SELECT 
  dicom
, PatientID
, StudyID
, StudyDate
, StudyTime 
FROM `physionet-data.mimic_cxr.dicom_metadata_string` 
ORDER BY PatientID, StudyDate

'''
mimic_cxr = run_query(query)

mimic_cxr.head(5)

In [ ]:
mimic_cxr.to_csv('mimic_cxr.csv')
files.download('mimic_cxr.csv')

In [ ]:
query = '''
with mv as (
SELECT distinct stay_id FROM `physionet-data.mimiciv_derived.ventilation` 
where ventilation_status='InvasiveVent'
)

select subject_id, hadm_id, ics.stay_id
FROM `physionet-data.mimiciv_icu.icustays` ics, mv
where ics.stay_id=mv.stay_id

ORDER BY subject_id, ics.stay_id
'''
derived_MV = run_query(query)
derived_MV.head()

In [ ]:
derived_MV.to_csv('derived_MV.csv')
files.download('derived_MV.csv')

In [ ]:
query= '''
with ic as
(select subject_id, hadm_id, stay_id
FROM `physionet-data.mimiciv_icu.icustays`)

SELECT pO2.subject_id, po2, charttime
FROM `physionet-data.mimiciv_derived.bg` pO2, ic
WHERE SPECIMEN= 'ART.' AND ic.subject_id = pO2.subject_id
ORDER BY subject_id
'''
derived_pO2 = run_query(query)
derived_pO2.head()

In [ ]:
derived_pO2.to_csv('derived_pO2.csv')
files.download('derived_pO2.csv')

In [ ]:
query= '''
SELECT subject_id, fio2_chartevents, charttime,
    CASE
      WHEN fio2_chartevents >= 21.0 THEN fio2_chartevents * 0.01
      WHEN fio2_chartevents >= 1.0 and fio2_chartevents < 21.0 THEN (20 + (4 * fio2_chartevents)) * 0.01
      WHEN fio2_chartevents < 1.0 AND fio2_chartevents > 0 THEN fio2_chartevents
      WHEN fio2_chartevents = 0.0 THEN 0.21
      ELSE NULL
    END as converted_fiO2,
FROM `physionet-data.mimiciv_derived.bg` 
WHERE SPECIMEN= 'ART.'
ORDER BY subject_id
'''
derived_fiO2 = run_query(query)
derived_fiO2.head()

In [ ]:
derived_fiO2.to_csv('derived_fiO2.csv')
files.download('derived_fiO2.csv')

In [ ]:
query = ''' 

 SELECT 
   subject_id
   , hadm_id
    , stay_id
    , los
  FROM physionet-data.mimiciv_icu.icustays 
ORDER BY subject_id, hadm_id, stay_id
'''
icu_los = run_query(query)

icu_los.head(5)

In [ ]:
icu_los.to_csv('icu_los.csv')
files.download('icu_los.csv')

In [ ]:
query = ''' 

 SELECT 
   subject_id
   , hadm_id

    , hospital_expire_flag
  FROM physionet-data.mimiciv_hosp.admissions
ORDER BY subject_id, hadm_id
'''
hosp_mort = run_query(query)

hosp_mort.head(5)

In [ ]:
hosp_mort.to_csv('hosp_mort.csv')
files.download('hosp_mort.csv')

In [ ]:
query = '''
with height as (
SELECT distinct stay_id, height FROM `physionet-data.mimiciv_derived.height` 
)

select subject_id, hadm_id, ics.stay_id
FROM `physionet-data.mimiciv_icu.icustays` ics, mv
where ics.stay_id=mv.stay_id

ORDER BY subject_id, ics.stay_id
'''
derived_MV = run_query(query)
derived_MV.head()

In [ ]:
query = ''' 
with ethnicity as(
  SELECT 
  subject_id,
  race,
  FROM physionet-data.mimiciv_hosp.admissions
  )

SELECT
pts.subject_id, pts.gender,
ethnicity.race
FROM physionet-data.mimiciv_hosp.patients pts
INNER JOIN ethnicity
ON ethnicity.subject_id = pts.subject_id
'''
bmi = run_query(query)

bmi.head(5)

In [ ]:
gender_ethnicity = bmi
gender_ethnicity.to_csv('gender_ethnicity.csv')
files.download('gender_ethnicity.csv')

In [ ]:
query = ''' 
 SELECT 
   subject_id, 
   chartdate,
   result_value, 

  FROM physionet-data.mimiciv_hosp.omr
  WHERE result_name = 'BMI (kg/m2)'
'''
bmi = run_query(query)

bmi.head(5)

In [ ]:
bmi.to_csv('bmi.csv')
files.download('bmi.csv')

In [ ]:
query = ''' 
 SELECT 
   subject_id, 
   hadm_id,
   stay_id, 
   intime, outtime,
   los

  FROM physionet-data.mimiciv_icu.icustays
  ORDER BY subject_id, hadm_id, stay_id, intime
'''
stay_id = run_query(query)

stay_id.head(5)

In [ ]:
stay_id.to_csv('stay_id.csv')
files.download('stay_id.csv')

In [ ]:
query = ''' 
 SELECT 
   subject_id, 
   stay_id, 
   charttime,
   respiratory_rate_total,
   minute_volume,
   tidal_volume_observed,
   plateau_pressure,
   peep,
   fio2,
   ventilator_mode

  FROM physionet-data.mimiciv_derived.ventilator_setting
  ORDER BY subject_id, stay_id, charttime
'''
physiology = run_query(query)

physiology.head(5)

In [ ]:
physiology.to_csv('physiology.csv')
files.download('physiology.csv')

In [ ]:
query = ''' 

SELECT
oasis.subject_id, 
oasis.hadm_id,
oasis.stay_id,
oasis.oasis,
oasis.oasis_prob,

FROM physionet-data.mimiciv_derived.oasis oasis
'''
oasis_score = run_query(query)

oasis_score.head(5)

In [ ]:
oasis_score.to_csv('oasis_scores.csv')
files.download('oasis_scores.csv')

In [ ]:

query = ''' 

  SELECT 
*
  FROM physionet-data.mimiciv_derived.apsiii
'''
apsiii_score = run_query(query)

apsiii_score.head(5)

In [ ]:
apsiii_score.to_csv('apsiii_score.csv')
files.download('apsiii_score.csv')

In [ ]:

query = ''' 

  SELECT 
stay_id,
starttime,
endtime
  FROM physionet-data.mimiciv_derived.ventilation
  WHERE ventilation_status='InvasiveVent'
'''
lomv = run_query(query)

lomv.head(5)

In [ ]:
lomv.to_csv('lomv.csv')
files.download('lomv.csv')

In [ ]:
query='''

SELECT
w.stay_id,
starttime,
weight,
weight_type,

FROM physionet-data.mimiciv_derived.weight_durations w

ORDER BY w.stay_id
'''
hw = run_query(query)
hw.head(5)

In [ ]:
hw.to_csv('weight.csv')
files.download('weight.csv')

In [ ]:
query='''

  SELECT
*

  FROM physionet-data.mimiciv_derived.height

ORDER BY stay_id
'''
height = run_query(query)
height.head(5)

In [ ]:
height.to_csv('height.csv')
files.download('height.csv')

In [ ]:
query='''

  SELECT
*

  FROM physionet-data.mimiciv_derived.charlson

ORDER BY subject_id, hadm_id
'''
comorb = run_query(query)
comorb.head(5)

In [ ]:
comorb.to_csv('comorb.csv')
files.download('comorb.csv')

In [ ]:
query = '''
With icd as
(SELECT
icd_code, long_title

FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`)

SELECT
di.hadm_id,
icd.long_title
From physionet-data.mimiciv_hosp.diagnoses_icd di
INNER JOIN icd
ON di.icd_code = icd.icd_code

'''

diagnoses = run_query(query)
diagnoses.head(5)

In [ ]:
diagnoses.to_csv('diagnoses.csv')
files.download('diagnoses.csv')

In [ ]:
query= '''
WITH diag AS (
    SELECT
        hadm_id
        , CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code
        , CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
)

, com AS (
    SELECT
        ad.hadm_id

        -- Myocardial infarction
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('410', '412')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I21', 'I22')
            OR
            SUBSTR(icd10_code, 1, 4) = 'I252'
            THEN 1
            ELSE 0 END) AS myocardial_infarct

        -- Congestive heart failure
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '428'
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN ('39891', '40201', '40211', '40291', '40401', '40403'
                , '40411', '40413', '40491', '40493')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4254' AND '4259'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I43', 'I50')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('I099', 'I110', 'I130', 'I132', 'I255', 'I420'
                  , 'I425', 'I426', 'I427', 'I428', 'I429', 'P290'
            )
            THEN 1
            ELSE 0 END) AS congestive_heart_failure

        -- Peripheral vascular disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('440', '441')
            OR
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('0930', '4373', '4471', '5571', '5579', 'V434')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4431' AND '4439'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I70', 'I71')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I731', 'I738', 'I739', 'I771', 'I790'
                                         , 'I792'
                                         , 'K551'
                                         , 'K558'
                                         , 'K559'
                                         , 'Z958'
                                         , 'Z959'
            )
            THEN 1
            ELSE 0 END) AS peripheral_vascular_disease

        -- Cerebrovascular disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '430' AND '438'
            OR
            SUBSTR(icd9_code, 1, 5) = '36234'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('G45', 'G46')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'I60' AND 'I69'
            OR
            SUBSTR(icd10_code, 1, 4) = 'H340'
            THEN 1
            ELSE 0 END) AS cerebrovascular_disease

        -- Dementia
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '290'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('2941', '3312')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('F00', 'F01', 'F02', 'F03', 'G30')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('F051', 'G311')
            THEN 1
            ELSE 0 END) AS dementia

        -- Chronic pulmonary disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '490' AND '505'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4168', '4169', '5064', '5081', '5088')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J40' AND 'J47'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J60' AND 'J67'
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I278', 'I279', 'J684', 'J701', 'J703')
            THEN 1
            ELSE 0 END) AS chronic_pulmonary_disease

        -- Rheumatic disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '725'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4465', '7100', '7101', '7102', '7103'
                                        , '7104', '7140', '7141', '7142', '7148'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('M05', 'M06', 'M32', 'M33', 'M34')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('M315', 'M351', 'M353', 'M360')
            THEN 1
            ELSE 0 END) AS rheumatic_disease

        -- Peptic ulcer disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('531', '532', '533', '534')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('K25', 'K26', 'K27', 'K28')
            THEN 1
            ELSE 0 END) AS peptic_ulcer_disease

        -- Mild liver disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('570', '571')
            OR
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('0706', '0709', '5733', '5734', '5738', '5739', 'V427')
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN ('07022', '07023', '07032', '07033', '07044', '07054')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('B18', 'K73', 'K74')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('K700', 'K701', 'K702', 'K703', 'K709', 'K713'
                  , 'K714', 'K715', 'K717', 'K760', 'K762'
                  , 'K763', 'K764', 'K768', 'K769', 'Z944')
            THEN 1
            ELSE 0 END) AS mild_liver_disease

        -- Diabetes without chronic complication
        , MAX(CASE WHEN
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('2500', '2501', '2502', '2503', '2508', '2509')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('E100', 'E101', 'E106', 'E108', 'E109', 'E110', 'E111'
                  , 'E116'
                  , 'E118'
                  , 'E119'
                  , 'E120'
                  , 'E121'
                  , 'E126'
                  , 'E128'
                  , 'E129'
                  , 'E130'
                  , 'E131'
                  , 'E136'
                  , 'E138'
                  , 'E139'
                  , 'E140'
                  , 'E141', 'E146', 'E148', 'E149')
            THEN 1
            ELSE 0 END) AS diabetes_without_cc

        -- Diabetes with chronic complication
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('2504', '2505', '2506', '2507')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('E102', 'E103', 'E104', 'E105', 'E107', 'E112', 'E113'
                  , 'E114'
                  , 'E115'
                  , 'E117'
                  , 'E122'
                  , 'E123'
                  , 'E124'
                  , 'E125'
                  , 'E127'
                  , 'E132'
                  , 'E133'
                  , 'E134'
                  , 'E135'
                  , 'E137'
                  , 'E142'
                  , 'E143', 'E144', 'E145', 'E147')
            THEN 1
            ELSE 0 END) AS diabetes_with_cc

        -- Hemiplegia or paraplegia
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('342', '343')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('3341', '3440', '3441', '3442'
                                        , '3443', '3444', '3445', '3446', '3449'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('G81', 'G82')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('G041', 'G114', 'G801', 'G802', 'G830'
                                         , 'G831'
                                         , 'G832'
                                         , 'G833'
                                         , 'G834'
                                         , 'G839'
            )
            THEN 1
            ELSE 0 END) AS paraplegia

        -- Renal disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('582', '585', '586', 'V56')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('5880', 'V420', 'V451')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5830' AND '5837'
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN (
                '40301'
                , '40311'
                , '40391'
                , '40402'
                , '40403'
                , '40412'
                , '40413'
                , '40492'
                , '40493'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('N18', 'N19')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I120', 'I131', 'N032', 'N033', 'N034'
                                         , 'N035'
                                         , 'N036'
                                         , 'N037'
                                         , 'N052'
                                         , 'N053'
                                         , 'N054'
                                         , 'N055'
                                         , 'N056'
                                         , 'N057'
                                         , 'N250'
                                         , 'Z490'
                                         , 'Z491'
                                         , 'Z492'
                                         , 'Z940'
                                         , 'Z992'
            )
            THEN 1
            ELSE 0 END) AS renal_disease

        -- Any malignancy, including lymphoma and leukemia,
        -- except malignant neoplasm of skin.
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '140' AND '172'
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '1740' AND '1958'
            OR
            SUBSTR(icd9_code, 1, 3) BETWEEN '200' AND '208'
            OR
            SUBSTR(icd9_code, 1, 4) = '2386'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('C43', 'C88')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C00' AND 'C26'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C30' AND 'C34'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C37' AND 'C41'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C45' AND 'C58'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C60' AND 'C76'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C81' AND 'C85'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C90' AND 'C97'
            THEN 1
            ELSE 0 END) AS malignant_cancer

        -- Moderate or severe liver disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('4560', '4561', '4562')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5722' AND '5728'
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('I850', 'I859', 'I864', 'I982', 'K704', 'K711'
                  , 'K721', 'K729', 'K765', 'K766', 'K767')
            THEN 1
            ELSE 0 END) AS severe_liver_disease

        -- Metastatic solid tumor
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('196', '197', '198', '199')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('C77', 'C78', 'C79', 'C80')
            THEN 1
            ELSE 0 END) AS metastatic_solid_tumor

        -- AIDS/HIV
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('042', '043', '044')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('B20', 'B21', 'B22', 'B24')
            THEN 1
            ELSE 0 END) AS aids
    FROM `physionet-data.mimiciv_hosp.admissions` ad
    LEFT JOIN diag
        ON ad.hadm_id = diag.hadm_id
    GROUP BY ad.hadm_id
)

, ag AS (
    SELECT
        hadm_id
        , age
        , CASE WHEN age <= 50 THEN 0
            WHEN age <= 60 THEN 1
            WHEN age <= 70 THEN 2
            WHEN age <= 80 THEN 3
            ELSE 4 END AS age_score
    FROM `physionet-data.mimiciv_derived.age`
)

SELECT
    ad.subject_id
    , ad.hadm_id
    , ag.age_score
    , myocardial_infarct
    , congestive_heart_failure
    , peripheral_vascular_disease
    , cerebrovascular_disease
    , dementia
    , chronic_pulmonary_disease
    , rheumatic_disease
    , peptic_ulcer_disease
    , mild_liver_disease
    , diabetes_without_cc
    , diabetes_with_cc
    , paraplegia
    , renal_disease
    , malignant_cancer
    , severe_liver_disease
    , metastatic_solid_tumor
    , aids
    -- Calculate the Charlson Comorbidity Score using the original
    -- weights from Charlson, 1987.
    , age_score
    + myocardial_infarct + congestive_heart_failure
    + peripheral_vascular_disease + cerebrovascular_disease
    + dementia + chronic_pulmonary_disease
    + rheumatic_disease + peptic_ulcer_disease
    + GREATEST(mild_liver_disease, 3 * severe_liver_disease)
    + GREATEST(2 * diabetes_with_cc, diabetes_without_cc)
    + GREATEST(2 * malignant_cancer, 6 * metastatic_solid_tumor)
    + 2 * paraplegia + 2 * renal_disease
    + 6 * aids
    AS charlson_comorbidity_index
FROM `physionet-data.mimiciv_hosp.admissions` ad
LEFT JOIN com
    ON ad.hadm_id = com.hadm_id
LEFT JOIN ag
    ON com.hadm_id = ag.hadm_id

'''

comorb = run_query(query)
comorb.head(5)

In [ ]:
comorb.to_csv('comorb2.csv')
files.download('comorb2.csv')

In [ ]:
query= '''
SELECT ie.subject_id, ie.hadm_id, ie.stay_id

    -- patient level factors
    , pat.gender, pat.dod

    -- hospital level factors
    , adm.admittime, adm.dischtime
    , DATETIME_DIFF(adm.dischtime, adm.admittime, DAY) AS los_hospital
    -- calculate the age as anchor_age (60) plus difference between
    -- admit year and the anchor year.
    -- the noqa retains the extra long line so the 
    -- convert to postgres bash script works
    , pat.anchor_age + DATETIME_DIFF(adm.admittime, DATETIME(pat.anchor_year, 1, 1, 0, 0, 0), YEAR) AS admission_age -- noqa: L016
    , adm.race
    , adm.hospital_expire_flag
    , DENSE_RANK() OVER (
        PARTITION BY adm.subject_id ORDER BY adm.admittime
    ) AS hospstay_seq
    , CASE
        WHEN
            DENSE_RANK() OVER (
                PARTITION BY adm.subject_id ORDER BY adm.admittime
            ) = 1 THEN True
        ELSE False END AS first_hosp_stay

    -- icu level factors
    , ie.intime AS icu_intime, ie.outtime AS icu_outtime
    , ROUND(
        CAST(DATETIME_DIFF(ie.outtime, ie.intime, HOUR) / 24.0 AS NUMERIC), 2
    ) AS los_icu
    , DENSE_RANK() OVER (
        PARTITION BY ie.hadm_id ORDER BY ie.intime
    ) AS icustay_seq

    -- first ICU stay *for the current hospitalization*
    , CASE
        WHEN
            DENSE_RANK() OVER (
                PARTITION BY ie.hadm_id ORDER BY ie.intime
            ) = 1 THEN True
        ELSE False END AS first_icu_stay

FROM `physionet-data.mimiciv_icu.icustays` ie
INNER JOIN `physionet-data.mimiciv_hosp.admissions` adm
    ON ie.hadm_id = adm.hadm_id
INNER JOIN `physionet-data.mimiciv_hosp.patients` pat
    ON ie.subject_id = pat.subject_id

'''

icu_detail= run_query(query)
icu_detail.head(5)

In [ ]:
icu_detail.to_csv('icu_detail.csv')
files.download('icu_detail.csv')

In [ ]:
query = ''' 

  SELECT 
  stay_id, starttime, sofa_24hours
  FROM physionet-data.mimiciv_derived.sofa

'''
sofa_score = run_query(query)

sofa_score.head(5)

In [ ]:
sofa_score.to_csv('sofa_score.csv')
files.download('sofa_score.csv')

In [ ]:
query = ''' 

  SELECT 
sirs.stay_id, sirs.sirs
  FROM physionet-data.mimiciv_derived.sirs sirs
'''
sirs_score = run_query(query)

sirs_score.head(5)

In [ ]:
sirs_score.to_csv('sirs_score.csv')
files.download('sirs_score.csv')

In [ ]:
query = ''' 

  SELECT 
*
  FROM `physionet-data.mimiciv_derived.vasoactive_agent`
'''
vasoactive = run_query(query)

vasoactive.head(5)

In [ ]:
vasoactive.to_csv('vasoactive.csv')
files.download('vasoactive.csv')

In [ ]:
query = ''' 

  SELECT 
*
  FROM `physionet-data.mimiciv_derived.rrt`
'''
rrt = run_query(query)

rrt.head(5)

In [ ]:
rrt.to_csv('rrt.csv')
files.download('rrt.csv')